In [1]:
import matplotlib.pyplot as plt
import numpy as np
from vpython import *
from sympy import *
def column(matrix, i):
    return [row[i] for row in matrix]

<IPython.core.display.Javascript object>

In [2]:
def com(points):
    x = column(points, 0)
    y = column(points, 1)
    z = column(points, 2)
    xavg = np.mean(x)
    yavg = np.mean(y)
    zavg = np.mean(z)
    cm = [xavg, yavg, zavg]
    return cm

In [3]:
def color(t):
    val = 1 - 0.5**abs(t)
    if t>0:
        col = vector(val,0,0)
    elif t<0:
        col = vector(0,0,val)
    elif t==0:
        col = vector(0.5, 0, 0.5)
    return col

In [4]:
def alexpolynomial(x,y,z):
    points = np.zeros(shape=(len(x),3))
    points[:,0]=x
    points[:,1]=y
    points[:,2]=z

    #Remove extraneous 3D points
    deletes = []
    for i in range(0,len(points)-1):
        if (points[i][0]==points[i+1][0]) and (points[i][1]==points[i+1][1]):
            deletes.append(i+1)       
    project = np.delete(points, deletes, axis=0)

    #Finding the corssing points
    num = 0
    oindex = []
    uindex = []
    for i in range(1,len(project)-1):
        point1 = project[i]
        for j in range(i+1,len(project)):
            point2 = project[j]
            if (point1[0]==point2[0]) and (point1[1]==point2[1]):
                num = num + 1
                if (point1[2]>point2[2]):
                    oindex.append(i)
                    uindex.append(j)
                else:
                    oindex.append(j)
                    uindex.append(i)

    project = np.delete(project, 2, axis=1)   #reduce to the string to a 2D projection             
    t = symbols('t')
    M = zeros(num, num) #Create empty matrix for alexander polynomial

    #Iterate over the number of crossings
    for i in range(num):
        over = project[int(oindex[i]+1)] - project[int(oindex[i])]
        under = project[int(uindex[i]+1)] - project[int(uindex[i])]
        if np.cross(over,under) == -1:
            pol = [1-t, t, -1] #order is: [overstrand arc, understrand head arc, understrand tail arc]
        else:
            pol = [1-t, -1, t]

        endarc = [a-1 for a in uindex] #define the arc by the end of each arc, i.e. the point where it is cut off 
                                    #by the overstrand at the crossing. Then the 'name' of the arc is the index.

        #for overstrand arc
        list1 = endarc + [oindex[i]]
        list1 = sorted(list1)
        arc1 = list1.index(int(oindex[i]))
        if arc1 > (num-1):
            arc1 = 0
        M[i*(num) + arc1] = pol[0]

        #for understrand head arc
        list2 = endarc + [uindex[i]+1]
        list2 = sorted(list2)
        arc2 = list2.index(int(uindex[i]+1))
        if arc2 > (len(endarc)-1):
            arc2 = 0
        M[i*(num) + arc2] = pol[1]

        #for understrand tail arc
        list3 = sorted(endarc)
        arc3 = list3.index(int(uindex[i]-1))
        M[i*(num) + arc3] = pol[2]

    if num==0:
        M = Matrix([1])
    Mdupe = 1*M
    M.col_del(0)
    M.row_del(0)
    return M.det(), M, Mdupe

In [5]:
(x,y,z) = np.loadtxt('/Users/Keerthana/Downloads/Not Knots - Sheet1.csv', delimiter=',', unpack=True)

In [6]:
alex = alexpolynomial(column(points,0), column(points,1), column(points,2))[0]
display(alex)

TypeError: 'type' object is not iterable

In [6]:
scene = canvas()

<IPython.core.display.Javascript object>

In [7]:
points = []
trefoil = curve(color = vector(0.5,0,0.5))
trefoil.visible=False
grid = np.zeros(shape=(20, 20, 20, 20))
for i in range(0, len(x)):
    a = int(x[i])
    b = int(y[i])
    c = int(z[i])
    vec = vector(a,b,c)
    trefoil.append(vec)
    points.append([a,b,c,0])
    grid[a][b][c][0] = 1
    
trefoil.visible=True

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [8]:
timelim = 10
trefoil.radius=0.07

alex = alexpolynomial(column(points,0), column(points,1), column(points,2))[0]
display(alex)
t=0
time = [0]
length = len(x)-3
while (t<timelim) and (alex!=1):
    for j in range(length, 0, -1):
        rate(20)
        prev = points[j-1]
        cur = points[j]
        nex = points[j+1]
        nexx = points[j+2]
        s1 = np.subtract(cur,prev)
        s2 = np.subtract(nex,nexx)
        compare = s1 == s2
        if (compare.all()):
            s3 = np.subtract(nex, cur)
            perp1 = np.array([s3[1], s3[2], s3[3], s3[0]])
            perp2 = np.array([s3[2], s3[3], s3[0], s3[1]])
            perp3 = np.array([s3[3], s3[0], s3[1], s3[2]])
            perps = [perp1, perp2, perp3, -perp1, -perp2, -perp3]
            np.random.shuffle(perps)   
            curn = prev + perps[0]
            nexn = nexx + perps[0]
            
            if ((grid[curn[0]][curn[1]][curn[2]][curn[3]]==0) and (grid[nexn[0]][nexn[1]][nexn[2]][nexn[3]]==0)):
                points[j]=[curn[0], curn[1], curn[2], curn[3]]
                points[j+1] = [nexn[0], nexn[1], nexn[2], nexn[3]]
                grid[cur[0]][cur[1]][cur[2]][cur[3]]=0
                grid[nex[0]][nex[1]][nex[2]][nex[3]]=0
                grid[curn[0]][curn[1]][curn[2]][curn[3]]=1
                grid[nexn[0]][nexn[1]][nexn[2]][nexn[3]]=1
                
                P1 = trefoil.point(j)
                new1 = vector(curn[0], curn[1], curn[2])
                P1['pos'] = new1
                trefoil.splice(j,1,P1['pos'])
                col1 = color(curn[3])
                trefoil.modify(j, color = col1)
                
                P2 = trefoil.point(j+1)
                new2 = vector(nexn[0], nexn[1], nexn[2])
                P2['pos'] = new2
                trefoil.splice(j+1,1,P2['pos'])
                col2 = color(nexn[3])
                trefoil.modify(j+1, color = col2)
            else:
                continue
            
        x1 = points[j][0]
        y1 = points[j][1]
        z1 = points[j][2]
        w1 = points[j][3]
        prev = points[j-1]
        cur = points[j]
        nex = points[j+1]
        d1 = np.subtract(nex, cur)
        d2 = np.subtract(prev, cur)
        dot = np.multiply(d1,d2)
        compare = dot == [0,0,0,0]
        if (compare.all()):
            other = np.subtract(nex,cur) + prev
            x2 = other[0]
            y2 = other[1]
            z2 = other[2]
            w2 = other[3]
            if (grid[x2][y2][z2][w2]==0):
                rand = np.random.randint(1, 3, size=1)
                if (rand==1):
                    points[j]=[x2,y2,z2,w2]
                    grid[x2][y2][z2][w2]=1
                    grid[x1][y1][z1][w1]=0
                    P = trefoil.point(j)
                    new = vector(x2, y2, z2)
                    P['pos'] = new
                    trefoil.splice(j,1,P['pos'])
                    col = color(w2)
                    trefoil.modify(j,color=col)
                else:
                    continue
            else:
                continue
        else:
            continue
            
    time.append(t+1)
    alex = alexpolynomial(column(points,0), column(points,1), column(points,2))[0]
    display(alex)
    if (alex==0):
        Mdupe = alexpolynomial(column(points,0), column(points,1), column(points,2))[2]
        display(Mdupe)
    t=t+1

-t**4 + t**3 - t**2

-t**2 + t - 1

0

Matrix([
[1 - t,     0, t,    -1, 0],
[1 - t,     t, 0,    -1, 0],
[    t,    -1, 0, 1 - t, 0],
[   -1, 1 - t, 0,     t, 0],
[   -1, 1 - t, 0,     0, t]])

t*(-t**3 + t - 1)

...

Matrix([
[1 - t,  0,     0, t,    -1, 0],
[1 - t,  0,     t, 0,    -1, 0],
[    t, -1,     0, 0, 1 - t, 0],
[    0, -1,     t, 0,     0, 0],
[   -1,  0, 1 - t, 0,     t, 0],
[   -1,  0, 1 - t, 0,     0, t]])

-t**4 + t**3 - t**2

-t**4 + 2*t**3 - 2*t**2

t**5 - 2*t**4 + 2*t**3

-2*t**4 + 2*t**3 - t**2

In [77]:
timelim = 10
trefoil.radius=0.07
# scene.center = vector(np.mean(xvals), np.mean(yvals), np.mean(zvals))

alex = alexpolynomial(column(points,0), column(points,1), column(points,2))
display(alex)
t=0
time = [0]
length = len(x)-3
while (t<timelim) and (alex!=1):
    for j in range(length, 0, -1):
        rate(20)
        x1 = points[j][0]
        y1 = points[j][1]
        z1 = points[j][2]
        w1 = points[j][3]
        prev = points[j-1]
        cur = points[j]
        nex = points[j+1]
        nexx = points[j+2]
        s1 = np.subtract(cur,prev)
        s2 = np.subtract(nex,nexx)
        compare = s1 == s2
        if (compare.all()):
            rand = np.random.randint(1, 7, size=1)
            cross = np.cross(s1[0:3],np.subtract(cur[0:3],nex[0:3]))
            a = cross == [0,0,0]
            if (a.all()):
                continue
            else:
                if (rand==1):
                    continue
                if (rand==2):
                    thing = -s1
                elif (rand == 3):
                    thing = [*cross, *[0]]
                elif (rand==4):
                    thing = [*(-cross), *[0]]
                elif (rand==5):
                    thing = [0,0,0,1]
                elif (rand==6):
                    thing = [0,0,0,-1]
                    
            print(rand, thing)   
            curn = prev + thing
            nexn = nexx + thing
            
            
            if ((grid[curn[0]][curn[1]][curn[2]][curn[3]]==0) and (grid[nexn[0]][nexn[1]][nexn[2]][nexn[3]]==0)):
                points[j]=[curn[0], curn[1], curn[2], curn[3]]
                points[j+1] = [nexn[0], nexn[1], nexn[2], nexn[3]]
                grid[cur[0]][cur[1]][cur[2]][cur[3]]=0
                grid[nex[0]][nex[1]][nex[2]][nex[3]]=0
                grid[curn[0]][curn[1]][curn[2]][curn[3]]=1
                grid[nexn[0]][nexn[1]][nexn[2]][nexn[3]]=1
                
                P1 = trefoil.point(j)
                new1 = vector(curn[0], curn[1], curn[2])
                P1['pos'] = new1
                trefoil.splice(j,1,P1['pos'])
                col1 = color(curn[3])
                trefoil.modify(j, color = col1)
                
                P2 = trefoil.point(j+1)
                new2 = vector(nexn[0], nexn[1], nexn[2])
                P2['pos'] = new2
                trefoil.splice(j+1,1,P2['pos'])
                col2 = color(nexn[3])
                trefoil.modify(j+1, color = col2)
            else:
                continue
            
        x1 = points[j][0]
        y1 = points[j][1]
        z1 = points[j][2]
        w1 = points[j][3]
        prev = points[j-1]
        cur = points[j]
        nex = points[j+1]
        d1 = np.subtract(nex, cur)
        d2 = np.subtract(prev, cur)
        dot = np.multiply(d1,d2)
        compare = dot == [0,0,0,0]
        if (compare.all()):
            other = np.subtract(nex,cur) + prev
            x2 = other[0]
            y2 = other[1]
            z2 = other[2]
            w2 = other[3]
            if (grid[x2][y2][z2][w2]==0):
                rand = np.random.randint(1, 3, size=1)
                if (rand==1):
                    points[j]=[x2,y2,z2,w2]
                    grid[x2][y2][z2][w2]=1
                    grid[x1][y1][z1][w1]=0
                    P = trefoil.point(j)
                    new = vector(x2, y2, z2)
                    P['pos'] = new
                    trefoil.splice(j,1,P['pos'])
                    col = color(w2)
                    trefoil.modify(j,color=col)
                else:
                    continue
            else:
                continue
        else:
            continue
            
    time.append(t+1)
    alex = alexpolynomial(column(points,0), column(points,1), column(points,2))
    t=t+1
    display(alex)

-t**4 + t**3 - t**2

-t**4 + t**3 - t**2

[2] [1 0 0 0]


0

[2] [0 1 0 0]


...

0

[6] [0, 0, 0, -1]
[5] [0, 0, 0, 1]
[5] [0, 0, 0, 1]
[3] [0, 0, 1, 0]


2*t**5 - 3*t**4 + 2*t**3

[6] [0, 0, 0, -1]
[5] [0, 0, 0, 1]
[5] [0, 0, 0, 1]
[3] [0, 0, 1, 0]
[6] [0, 0, 0, -1]
[2] [0 1 0 0]


-t**7 + t**6 + t**5 - 2*t**4

In [13]:

#trefoil.radius=0.07
trefoil.visible=True
scene.center = vector(-2,1,0)


t=0
while t<30:
    for j in range(49, 0, -1):
        rate(20)
        x1 = tpoints[j][0]
        y1 = tpoints[j][1]
        z1 = tpoints[j][2]
        prev = tpoints[j-1]
        cur = tpoints[j]
        nex = tpoints[j+1]
        d1 = np.subtract(nex, cur)
        d2 = np.subtract(prev, cur)
        dot = np.multiply(d1,d2)
        compare = dot == [0,0,0]
        if (compare.all()):
            other = np.subtract(nex,cur) + prev
            x2 = other[0]
            y2 = other[1]
            z2 = other[2]
            if (tgrid[x2][y2][z2]==0):
                rand = np.random.randint(1, 3, size=1)
                if (rand==1):
                    tpoints[j]=[x2,y2,z2]
                    tgrid[x2][y2][z2]=1
                    tgrid[x1][y1][z1]=0
                    P = trefoil.point(j)
                    new = vector(x2, y2, z2)
                    P['pos'] = new
                    trefoil.splice(j,1,P['pos'])
                else:
                    continue
            else:
                continue
        else:
            continue
    t=t+1
